In [3]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

# Import 
import json
import os
from collections import OrderedDict
import pandas as pd
import random
import torch
import torchvision
from torch import nn
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
!pip install ViT
!pip install vit-pytorch
from vit_pytorch import ViT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 17.1 MB/s eta 0:00:00


In [4]:
data_dir = '/notebooks/flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
#!unzip /flowers.zip

In [5]:
# TODO: Define your transforms for the training, validation, and testing sets
#data_transforms = # 224 -> 256
train_dir_transforms = transforms.Compose([transforms.RandomResizedCrop(224), # before all 225 ##256
                                           transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
valid_dir_transforms = transforms.Compose([transforms.Resize(224),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
test_dir_transforms = transforms.Compose([transforms.Resize(224),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
#image_datasets = 
train_data = ImageFolder(train_dir, train_dir_transforms)
valid_set = ImageFolder(valid_dir, valid_dir_transforms)
test_set = ImageFolder(test_dir, test_dir_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
#dataloaders = 
trainloader = torch.utils.data.DataLoader(train_data, batch_size =64, shuffle=True)
valid_data = torch.utils.data.DataLoader(valid_set, batch_size=64)
test_data = torch.utils.data.DataLoader(test_set, batch_size=64)

In [6]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [7]:
trainloader.dataset[222]

(tensor([[[-1.7240, -1.7240, -1.7069,  ..., -1.8953, -1.8953, -1.8610],
          [-1.7583, -1.7412, -1.7412,  ..., -1.9124, -1.9124, -1.8953],
          [-1.7925, -1.7754, -1.7754,  ..., -1.9295, -1.9295, -1.9124],
          ...,
          [-1.2274, -1.2617, -1.3130,  ..., -1.6042, -1.2959, -1.0219],
          [-1.2274, -1.2959, -1.3473,  ..., -1.4329, -1.0390, -0.6794],
          [-1.2445, -1.3130, -1.3473,  ..., -1.1932, -0.6965, -0.2856]],
 
         [[-1.2129, -1.2129, -1.1954,  ..., -1.5630, -1.5630, -1.5980],
          [-1.2654, -1.2479, -1.2479,  ..., -1.5980, -1.5980, -1.6331],
          [-1.3179, -1.3004, -1.3004,  ..., -1.6331, -1.6331, -1.6506],
          ...,
          [-0.2500, -0.3025, -0.3725,  ..., -1.1078, -0.7577, -0.3901],
          [-0.2325, -0.3025, -0.3901,  ..., -0.8978, -0.4951, -0.0399],
          [-0.2675, -0.3375, -0.4251,  ..., -0.6352, -0.1099,  0.3627]],
 
         [[-1.4036, -1.4036, -1.4210,  ..., -1.6302, -1.6302, -1.6127],
          [-1.4384, -1.4210,

In [45]:
"""
Create a visual transformer, declaring the number of classes, image size, etc.
Make sure that image_size is divisible by patch_size.
"""
v = ViT(
    image_size = 224,
    patch_size = 32, #32
    num_classes = 102,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.5,
#    emb_dropout = 0.1
)
model = ViT(
    image_size = 224,
    patch_size = 32, #32
    num_classes = 102,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.5,
#    emb_dropout = 0.1
)

In [22]:
def validation(model, valid_data, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in valid_data:
        
        images,labels = images.to('cuda'), labels.to('cuda')
        
        output = model.forward(images)
        batch_loss = criterion(output, labels)
        test_loss += batch_loss.item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)
criterion = nn.functional.nll_loss
epochs = 12
#running_loss = 0
model.to('cuda')
for epoch in range(epochs):
  model.train()
  counter = 0
  for data in trainloader:
    X , y= data
    X , y = X.to('cuda'), y.to('cuda')
    optimizer.zero_grad() # clear gradient information.
    #output = model(X) <- gives errors
    output = model.forward(X)
    loss = criterion(output, y)
    loss.backward() # do pack-propagation step
    optimizer.step() # tell optimizer that you finished batch/iteration.
    counter += 1
    #running_loss += loss.item()
 
 
  # network in eval mode for inference
  model.eval()
            
  # Turn off gradients for validation, saves memory and computations
  with torch.no_grad():
      test_loss, accuracy = validation(model, valid_data, criterion)
  print("Epoch: {}/{}...".format(epoch+1,epochs),
        "Loss: {:.4f}".format(loss.data),
        "Train Loss: {:.3f}.. ".format(test_loss/len(valid_data)),
        "Train Accuracy: {:.3f}".format(accuracy/len(valid_data)))
  running_loss = 0
  model.train()

Epoch: 1/12... Loss: -41.7776 Train Loss: -43.188..  Train Accuracy: 0.095
Epoch: 2/12... Loss: -50.1237 Train Loss: -47.142..  Train Accuracy: 0.090
Epoch: 3/12... Loss: -49.2122 Train Loss: -51.170..  Train Accuracy: 0.089
Epoch: 4/12... Loss: -54.2257 Train Loss: -55.430..  Train Accuracy: 0.089
Epoch: 5/12... Loss: -56.6199 Train Loss: -59.732..  Train Accuracy: 0.093
Epoch: 6/12... Loss: -62.3951 Train Loss: -64.114..  Train Accuracy: 0.094
Epoch: 7/12... Loss: -64.8251 Train Loss: -68.692..  Train Accuracy: 0.079
Epoch: 8/12... Loss: -68.2049 Train Loss: -73.238..  Train Accuracy: 0.079
Epoch: 9/12... Loss: -74.3376 Train Loss: -78.128..  Train Accuracy: 0.079
Epoch: 10/12... Loss: -82.5395 Train Loss: -82.982..  Train Accuracy: 0.060
Epoch: 11/12... Loss: -85.9194 Train Loss: -87.870..  Train Accuracy: 0.047
Epoch: 12/12... Loss: -86.6795 Train Loss: -92.888..  Train Accuracy: 0.047


In [54]:
# Evaluate the trained network.
total = 0
correct = 0
model.to('cuda')
with torch.no_grad():   # No need for keepnig track of necessary changes to the gradient.
  for X,y in test_data:
    X , y = X.to('cuda'), y.to('cuda')
    output = model(X)
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print('Accuracy:', round(correct/total, 3))

Accuracy: 0.104


In [38]:
import torch
optimizer = torch.optim.Adam(v.parameters(), lr=7.9)
for epocs in range(2):
  model.train()
  counter = 0
  for data in trainloader:
    #print(f"\r This is: {counter} of total: {data_length} ",end = " ")
    X , y= data
    X , y = X.to('cuda'), y.to('cuda')
    optimizer.zero_grad() # clear gradient information.
    output = model(X)
    loss = nn.functional.nll_loss(output, y)
    loss.backward() # do pack-propagation step
    optimizer.step() # tell optimizer that you finished batch/iteration.
    counter += 1
  print(loss.data)

tensor(-1.2461e+08, device='cuda:0')
tensor(-1.2016e+08, device='cuda:0')
